# AAGB TME1

### Remaster par Sharane K.Murali

In [174]:
import numpy as np
import pandas as pd

##  1. Needleman & Wunsch

L'objectif de cet exercice est de recoder l'algorithme de Needleman & Wunsch. Les différentes questions servent de guide à la mise en place de l'algorithme, mais ne doivent pas nécessairement être suivis à la lettre. Pour rappel, l'algorithme est le suivant :

* Initialisation :
$$
\begin{cases}
    S_{i,0} = i * g \\
    S_{j,0} = j * g
\end{cases}
$$

* Réccurence :
$$ S_{i,j} = max
\begin{cases}
    S_{i-1,j-1} + σ(a_{i},b_{j})\\
    S_{i-1,j} + g \\
    S_{i,j-1} + g
\end{cases}
$$  
  

* Traceback : à chaque étape, on garde en mémoire l'alignement qui donne le meilleur score. Cela donne le chemin qui donne le/les meilleurs alignements globaux.

1. Une matrice de distance est une matrice D de taille n, où n est la taille de l'alphabet. Etant donnée deux lettres de l'alphabet A et B, D(A, B) correspond à la distance entre les lettres A et B. Dans notre cas,l'alphabet peut être l'ensemble de nucléotides (A,C,G,T) ou l'ensemble des 20 acides aminés. La diagonale dela matrice D correspond à des matches, le reste à des mistmaches. Par exemple, pour une valeur de match de 1 et de mismatch de −1, D sera une matrice avec des 1 sur la diagonale et des -1 ailleurs. Ecrire une fonction renvoyant la matrice de distance, étant donnés un alphabet, un score de match et un score de mismatch.

In [175]:
#Fonction d'initialisation de la matrice de distance
def init_distance_matrix(alphabet,match,mismatch):
    
    #Récupération de la longueur de l'alphabet
    len_alph = len(alphabet)
    
    #Création de la matrice identié avec seulement la valeur de match sur la digonale
    dist_matrix = np.eye(len_alph,dtype=int) * match
    
    #Mise à jour des valeurs de mismatch
    dist_matrix[dist_matrix == 0] = mismatch
    
    return dist_matrix

2. Coder une fonction qui retourne la matrice de score (sans traceback) suivant l'algorithme de Needleman & Wunsch. Cette fonction peut par exemple prendre en entrée les deux séquences à aligner, la matrice de distance et une valeur correspondant au coût de gap. Quelle est la complexité de cet algorithme ?

In [176]:
#Fonction Needleman-Wunsch sans traceback
def NW_without_TB(S1,S2,init_mat,gap,alphabet):
    
    #Initialisation de la matrice de score
    len_s1 = len(S1)
    len_s2 = len(S2)
    
    mat_score = np.zeros((len_s1 + 1, len_s2 + 1), dtype=int)
    
    #Remplissage des gaps au niveau de l'indice 0 dans les deux dimensions
    mat_score[0, :] = np.arange(len_s2 + 1) * gap 
    mat_score[:, 0] = np.arange(len_s1 + 1) * gap
    
    #Remplissage de la matrice
    for i in range(1, len_s1 + 1):
        for j in range(1, len_s2 + 1):
            
            #Calcul de chaque condition
            diag = mat_score[i-1,j-1] + init_mat[alphabet.index(S1[i-1]),alphabet.index(S2[j-1])] 
            deletion = mat_score[i-1,j] + gap
            insertion = mat_score[i,j-1] + gap
            
            #Récupération du max
            mat_score[i,j] = max(diag,deletion,insertion)
            
    return mat_score


Réponse : La complexité est de O(len(s1) x len(s2))

3. Rajouter le traceback pour obtenir la fonction complète. La sortie de la fonction finale doit retourner l'alignement ainsi que le score associé. Etant données deux séquences, leur alignement est-il nécessairement unique ?

In [177]:
#Fonction Needleman-Wunsch avec traceback sans différence entre gaps
def NW_WO_extension(S1,S2,init_mat,gap,alphabet):
    
    #Initialisation de la matrice de score
    len_s1 = len(S1)
    len_s2 = len(S2)
    
    mat_score = np.zeros((len_s1 + 1, len_s2 + 1), dtype=int)
    
    #Remplissage des gaps au niveau de l'indice 0 dans les deux dimensions
    mat_score[0, :] = np.arange(len_s2 + 1) * gap 
    mat_score[:, 0] = np.arange(len_s1 + 1) * gap
    
    #Remplissage de la matrice
    for i in range(1, len_s1 + 1):
        for j in range(1, len_s2 + 1):
            
            #Calcul de chaque condition
            diag = mat_score[i-1,j-1] + init_mat[alphabet.index(S1[i-1]),alphabet.index(S2[j-1])] 
            deletion = mat_score[i-1,j] + gap
            insertion = mat_score[i,j-1] + gap
            
            #Récupération du max
            mat_score[i,j] = max(diag,deletion,insertion)
            
    #Traceback
    
    #Initialisation des listes et arrays nécessaires
    list_alignements = []
    alig_s1 = np.empty((0,),dtype=str)
    alig_s2 = np.empty((0,),dtype=str)
    
    l1 = len_s1
    l2 = len_s2
    
    #Parcours en reverse
    while l1 > 0 or l2 > 0:
        #Test si diagonale
        if l1 > 0 and l2 > 0 and mat_score[l1,l2] == mat_score[l1-1,l2-1] + init_mat[alphabet.index(S1[l1-1]),alphabet.index(S2[l2-1])]:
            alig_s1 = np.append(alig_s1,S1[l1-1])
            alig_s2 = np.append(alig_s2,S2[l2-1])
            l1 -= 1
            l2 -= 1
        
        #Test si délétion
        elif l1 > 0 and mat_score[l1,l2] == mat_score[l1-1,l2] + gap:
            alig_s1 = np.append(alig_s1,S1[l1-1])
            alig_s2 = np.append(alig_s2,'-')
            l1 -= 1
        
        #Test si insertion
        else :
            alig_s1 = np.append(alig_s1,'-')
            alig_s2 = np.append(alig_s2,S2[l2-1])
            l2 -= 1            
    
    #Renversement des arrays
    alig_s1 = np.flip(alig_s1)
    alig_s2 = np.flip(alig_s2)
    
    #Ajout à la liste des alignements
    list_alignements.append(''.join(alig_s1))
    list_alignements.append(''.join(alig_s2))

    return list_alignements, mat_score[len_s1,len_s2]
   

Réponse : Leur alignement peut ne pas être forcément unique en fonction du chemin pris par le traceback

4. En général, on considère que l'introduction d'un gap est moins couteuse que le prolongement d'un gap. Modifier l'algorithme afin de pouvoir prendre en compte des valeurs différents pour l'ouverture et le prolongement de gaps.

In [178]:
#Fonction Needleman-Wunsch avec ouverture et extension de gap
def NW(S1,S2,init_mat,ouverture,extension,alphabet):
    
    #Initialisation de la matrice de score
    len_s1 = len(S1)
    len_s2 = len(S2)
    
    mat_score = np.zeros((len_s1 + 1, len_s2 + 1), dtype=int)
    
    #Remplissage des gaps au niveau de l'indice 0 dans les deux dimensions
    mat_score[0, :] = np.arange(len_s2 + 1) * ouverture
    mat_score[:, 0] = np.arange(len_s1 + 1) * ouverture
    
    #Remplissage de la matrice
    for i in range(1, len_s1 + 1):
        for j in range(1, len_s2 + 1):
            #Calcul de chaque condition
            diag = mat_score[i-1,j-1] + init_mat[alphabet.index(S1[i-1]),alphabet.index(S2[j-1])] 
            deletion = mat_score[i-1,j] + (extension if i > 1 else ouverture)
            insertion = mat_score[i,j-1] + (extension if i > 1 else ouverture)
            
            #Récupération du max
            mat_score[i,j] = max(diag,deletion,insertion)
            
    #Traceback
    
    #Initialisation des listes et arrays nécessaires
    list_alignements = []
    alig_s1 = np.empty((0,),dtype=str)
    alig_s2 = np.empty((0,),dtype=str)
    
    l1 = len_s1
    l2 = len_s2
    
    #Parcours en reverse
    while l1 > 0 or l2 > 0:
        #Test si diagonale
        if l1 > 0 and l2 > 0 and mat_score[l1,l2] == mat_score[l1-1,l2-1] + init_mat[alphabet.index(S1[l1-1]),alphabet.index(S2[l2-1])]:
            alig_s1 = np.append(alig_s1,S1[l1-1])
            alig_s2 = np.append(alig_s2,S2[l2-1])
            l1 -= 1
            l2 -= 1
        
        #Test si délétion
        elif l1 > 0 and mat_score[l1,l2] == mat_score[l1-1,l2] + (extension if i > 1 else ouverture):
            alig_s1 = np.append(alig_s1,S1[l1-1])
            alig_s2 = np.append(alig_s2,'-')
            l1 -= 1
        
        #Test si insertion
        else :
            alig_s1 = np.append(alig_s1,'-')
            alig_s2 = np.append(alig_s2,S2[l2-1])
            l2 -= 1            
    
    #Renversement des arrays
    alig_s1 = np.flip(alig_s1)
    alig_s2 = np.flip(alig_s2)
    
    #Ajout à la liste des alignements
    list_alignements.append(''.join(alig_s1))
    list_alignements.append(''.join(alig_s2))

    return list_alignements, mat_score[len_s1,len_s2]
   

In [179]:
#Test complet 

#Variables globales
alphabet = ['A','C','G','T']
match = 1
mismatch = -1
gap = -1
ouverture = extension = gap
S1 = 'CATGAC'
S2 = 'TCTGAAC'

In [180]:
#Initialisation de la matrice de distance
init_mat = init_distance_matrix(alphabet,match,mismatch)

#Affichage en DataFrame
init_df = pd.DataFrame(init_mat,index = alphabet, columns = alphabet)

print(f"\nAffichage de la matrice de distance avec match = {match} et mismatch = {mismatch}\n")
init_df


Affichage de la matrice de distance avec match = 1 et mismatch = -1



,A,C,G,T
A,1,-1,-1,-1
C,-1,1,-1,-1
G,-1,-1,1,-1
T,-1,-1,-1,1


In [181]:
#Calul uniquement de la matrice de score
mat_score = NW_without_TB(S1,S2,init_mat,gap,alphabet)

#Affichage en DataFrame
list_s1 = [" "]+list(S1)
list_s2 = [" "]+list(S2)

nw_wo_tb_df = pd.DataFrame(mat_score,index = list_s1, columns = list_s2)

print(f"\nAffichage de la matrice de score pour NW sans traceback avec S1 = {S1}, S2 = {S2}, et gap = {gap}\n")
nw_wo_tb_df


Affichage de la matrice de score pour NW sans traceback avec S1 = CATGAC, S2 = TCTGAAC, et gap = -1



,,T,C,T,G,A,A,C
,0,-1,-2,-3,-4,-5,-6,-7
C,-1,-1,0,-1,-2,-3,-4,-5
A,-2,-2,-1,-1,-2,-1,-2,-3
T,-3,-1,-2,0,-1,-2,-2,-3
G,-4,-2,-2,-1,1,0,-1,-2
A,-5,-3,-3,-2,0,2,1,0
C,-6,-4,-2,-3,-1,1,1,2


In [182]:
#Exécution de NW
liste_align,score = NW(S1,S2,init_mat,ouverture,extension,alphabet)

print(f"\nAlignement obtenu après NW avec S1 = {S1}, S2 = {S2}, match = {match}, mismatch = {mismatch}, ouverture de gap = {ouverture} et extension de gap = {extension}\n")
print(f"Séquence 1 : {liste_align[0]}\nSéquence 2 : {liste_align[1]}\nScore : {score}")


Alignement obtenu après NW avec S1 = CATGAC, S2 = TCTGAAC, match = 1, mismatch = -1, ouverture de gap = -1 et extension de gap = -1

Séquence 1 : -CATG-AC
Séquence 2 : TC-TGAAC
Score : 2


##  2. Petit exemple

On aimerait voir ce que donne nos algorithmes sur des séquences protéiques réelles. On va prendre par exemple les protéines 2ABL et 1OPK, dont les séquences sont les suivantes:

• >2ABL:A|PDBID|CHAIN|SEQUENCE

MGPSENDPNLFVALYDFVASGDNTLSITKGEKLRVLGYNHNGEWCEAQTKNGQGWVPSNYITPVNSLEKHSWYHGPVSRNAAEYLLSSGINGSFLVRESESSPGQRSISLRYEGRVYHYRINTASDGKLYVSSESRFNTLAELVHHHSTVADGLITTLHYPAP

• >1OPK:A|PDBID|CHAIN|SEQUENCE

GAMDPSEALQRPVASDFEPQGLSEAARWNSKENLLAGPSENDPNLFVALYDFVASGDNTLSITKGEKLRVLGYNHNGEWCEAQTKNGQGWVPSNYITPVNSLEKHSWYHGPVSRNAAEYLLSSGINGSFLVRESESSPGQRSISLRYEGRVYHYRINTASDGKLYVSSESRFNTLAELVHHHSTVADGLITTLHYPAPKRNKPTIYGVSPNYDKWEMERTDITMKHKLGGGQYGEVYEGVWKKYSLTVAVKTLKEDTMEVEEFLKEAAVMKEIKHPNLVQLLGVCTREPPFYIITEFMTYGNLLDYLRECNRQEVSAVVLLYMATQISSAMEYLEKKNFIHRNLAARNCLVGENHLVKVADFGLSRLMTGDTYTAHAGAKFPIKWTAPESLAYNKFSIKSDVWAFGVLLWEIATYGMSPYPGIDLSQVYELLEKDYRMERPEGCPEKVYELMRACWQWNPSDRPSFAEIHQAFETMFQESSISDEVEKELGKRGT

1. Utiliser la matrice BLOSUM62 comme matrice de distance, avec ouverture de gap de coût 11 et extension de coût 1, pour aligner ces deux séquences avec l'algorithme de Needleman Wunsch.

In [192]:
#Lecture du fichier Blosum62
with open('BLOSUM62.txt','r') as file:
    
    #Récupération des lignes
    ligne = file.readlines()

#Récupération de l'alphabet
alphabet_aa = ligne[6].split()

#Récupération de la matrice
blosum62 = np.loadtxt(ligne[7:],skiprows =0, usecols = range(1,len(alphabet_aa)+1))

#Afiichage en DataFrame
blosum62_df = pd.DataFrame(blosum62,index = alphabet_aa, columns = alphabet_aa)
blosum62_df


,A,R,N,D,C,Q,E,G,H,I,...,P,S,T,W,Y,V,B,Z,X,*
A,4.0,-1.0,-2.0,-2.0,0.0,-1.0,-1.0,0.0,-2.0,-1.0,...,-1.0,1.0,0.0,-3.0,-2.0,0.0,-2.0,-1.0,0.0,-4.0
R,-1.0,5.0,0.0,-2.0,-3.0,1.0,0.0,-2.0,0.0,-3.0,...,-2.0,-1.0,-1.0,-3.0,-2.0,-3.0,-1.0,0.0,-1.0,-4.0
N,-2.0,0.0,6.0,1.0,-3.0,0.0,0.0,0.0,1.0,-3.0,...,-2.0,1.0,0.0,-4.0,-2.0,-3.0,3.0,0.0,-1.0,-4.0
D,-2.0,-2.0,1.0,6.0,-3.0,0.0,2.0,-1.0,-1.0,-3.0,...,-1.0,0.0,-1.0,-4.0,-3.0,-3.0,4.0,1.0,-1.0,-4.0
C,0.0,-3.0,-3.0,-3.0,9.0,-3.0,-4.0,-3.0,-3.0,-1.0,...,-3.0,-1.0,-1.0,-2.0,-2.0,-1.0,-3.0,-3.0,-2.0,-4.0
Q,-1.0,1.0,0.0,0.0,-3.0,5.0,2.0,-2.0,0.0,-3.0,...,-1.0,0.0,-1.0,-2.0,-1.0,-2.0,0.0,3.0,-1.0,-4.0
E,-1.0,0.0,0.0,2.0,-4.0,2.0,5.0,-2.0,0.0,-3.0,...,-1.0,0.0,-1.0,-3.0,-2.0,-2.0,1.0,4.0,-1.0,-4.0
G,0.0,-2.0,0.0,-1.0,-3.0,-2.0,-2.0,6.0,-2.0,-4.0,...,-2.0,0.0,-2.0,-2.0,-3.0,-3.0,-1.0,-2.0,-1.0,-4.0
H,-2.0,0.0,1.0,-1.0,-3.0,0.0,0.0,-2.0,8.0,-3.0,...,-2.0,-1.0,-2.0,-2.0,2.0,-3.0,0.0,0.0,-1.0,-4.0
I,-1.0,-3.0,-3.0,-3.0,-1.0,-3.0,-3.0,-4.0,-3.0,4.0,...,-3.0,-2.0,-1.0,-3.0,-1.0,3.0,-3.0,-3.0,-1.0,-4.0


In [193]:
#Variables
S_ABL = 'MGPSENDPNLFVALYDFVASGDNTLSITKGEKLRVLGYNHNGEWCEAQTKNGQGWVPSNYITPVNSLEKHSWYHGPVSRNAAEYLLSSGINGSFLVRESESSPGQRSISLRYEGRVYHYRINTASDGKLYVSSESRFNTLAELVHHHSTVADGLITTLHYPAP'
S_10PK = 'GAMDPSEALQRPVASDFEPQGLSEAARWNSKENLLAGPSENDPNLFVALYDFVASGDNTLSITKGEKLRVLGYNHNGEWCEAQTKNGQGWVPSNYITPVNSLEKHSWYHGPVSRNAAEYLLSSGINGSFLVRESESSPGQRSISLRYEGRVYHYRINTASDGKLYVSSESRFNTLAELVHHHSTVADGLITTLHYPAPKRNKPTIYGVSPNYDKWEMERTDITMKHKLGGGQYGEVYEGVWKKYSLTVAVKTLKEDTMEVEEFLKEAAVMKEIKHPNLVQLLGVCTREPPFYIITEFMTYGNLLDYLRECNRQEVSAVVLLYMATQISSAMEYLEKKNFIHRNLAARNCLVGENHLVKVADFGLSRLMTGDTYTAHAGAKFPIKWTAPESLAYNKFSIKSDVWAFGVLLWEIATYGMSPYPGIDLSQVYELLEKDYRMERPEGCPEKVYELMRACWQWNPSDRPSFAEIHQAFETMFQESSISDEVEKELGKRGT'

match_grand = 1
mismatch_grand = -1
ouverture_grand = -11
extension_grand = -1

init_mat_aa = init_distance_matrix(alphabet_aa,match_grand,mismatch_grand)

In [194]:
#Exécution de NW
grand_align,grand_score = NW(S_ABL,S_10PK,init_mat_aa,ouverture_grand,extension_grand,alphabet_aa)

print(f"\nAlignement obtenu après NW avec match = {match_grand}, mismatch = {mismatch_grand}, ouverture de gap = {ouverture_grand} et extension de gap = {extension_grand}\n")
print(f"Séquence 1 : {grand_align[0]}\n\nSéquence 2 : {grand_align[1]}\n\nScore : {grand_score}")


Alignement obtenu après NW avec match = 1, mismatch = -1, ouverture de gap = -11 et extension de gap = -1

Séquence 1 : MG-----------------------------------PSENDPNLFVALYDFVASGDNTLSITKGEKLRVLGYNHNGEWCEAQTKNGQGWVPSNYITPVNSLEKHSWYHGPVSRNAAEYLLSSGINGSFLVRESESSPGQRSISLRYEGRVYHYRINTASDGKLYVSSESRFNTLAELVHHHSTV-----------A-----------------------D-----------------G------------------------------------------L-----------I----T-------------------------T-----------------------------------------L--------H-----------------------------------------------------------Y--------P--------A---------P-------------------------------

Séquence 2 : GAMDPSEALQRPVASDFEPQGLSEAARWNSKENLLAGPSENDPNLFVALYDFVASGDNTLSITKGEKLRVLGYNHNGEWCEAQTKNGQGWVPSNYITPVNSLEKHSWYHGPVSRNAAEYLLSSGINGSFLVRESESSPGQRSISLRYEGRVYHYRINTASDGKLYVSSESRFNTLAELVHHHSTVADGLITTLHYPAPKRNKPTIYGVSPNYDKWEMERTDITMKHKLGGGQYGEVYEGVWKKYSLTVAVKTLKEDTMEVEEFLKEAAVMKEIKHPNLVQLLGVCTREPPFYIITEFMTYGNLLDYLRECNRQEVSAVVLLYMATQISSAMEYLEKKNFIHRNLAARNCLVGENHLVKVADFGLSRLMT

2. Comparer votre résultat avec l'algorithme Needleman Wunsch fourni par blast.

* Résultats fourni par BLAST

Query  2    GPSENDPNLFVALYDFVASGDNTLSITKGEKLRVLGYNHNGEWCEAQTKNGQGWVPSNYI  61  
Sbjct  37   GPSENDPNLFVALYDFVASGDNTLSITKGEKLRVLGYNHNGEWCEAQTKNGQGWVPSNYI  96

Query  62   TPVNSLEKHSWYHGPVSRNAAEYLLSSGINGSFLVRESESSPGQRSISLRYEGRVYHYRI  121  
Sbjct  97   TPVNSLEKHSWYHGPVSRNAAEYLLSSGINGSFLVRESESSPGQRSISLRYEGRVYHYRI  156

Query  122  NTASDGKLYVSSESRFNTLAELVHHHSTVADGLITTLHYPAP  163  
Sbjct  157  NTASDGKLYVSSESRFNTLAELVHHHSTVADGLITTLHYPAP  198  

On peut constater que plusieurs petits alignements sont obtenus par Blast alors que nous avons un grand alignement avec beaucoup de gaps.

3. Comparer les structures de ces deux protéines sur le site Protein Data Bank. Que constatez vous ?

2ABL se trouve chez l'Homme, 1OPK se trouve chez la souris et les deux correspondent à une Transférase/ ABL-Tyrosine-Kinase.


4. Sur quel algorithme se base BLAST général ?

Blast général se base sur un algorithme d'aligenment local.

##  3. Smith-Waterman (Bonus)

L'algorithme de Needleman & Wunsch est un alignement global. Il peut être plus utile de mettre en évidence des alignement locaux. Pour ce faire, on peut par exemple réutiliser l'algorithme précédent, mais en ajoutant les
modifications suivantes :

* Réccurence :
$$ S_{i,j} = max
\begin{cases}
    0\\
    S_{i-1,j-1} + σ(a_{i},b_{j})\\
    S_{i-1,j} + g \\
    S_{i,j-1} + g
\end{cases}
$$  
  

* Pour le traceback, au lieu de commencer en bas à droite du tableau, on part plutôt de la position qui donne le score le plus élevé.

1. Coder l'algorithme de Smith-Waterman. De la même façon, cet algorithme doit retourner l'alignement local entre les deux séquences données. On peut aussi retourner la localisation de cet alignement dans chaque séquence. Attention, il peut y avoir plusieurs maxima locaux : on peut au choix retourner un ou tous les meilleurs alignements.

In [206]:
#Fonction Smith-Waterman avec ouverture et extension de gap
def SW(S1,S2,init_mat,ouverture,extension,alphabet):
    
    #Initialisation de la matrice de score
    len_s1 = len(S1)
    len_s2 = len(S2)
    
    mat_score = np.zeros((len_s1 + 1, len_s2 + 1), dtype=int)
    mat_traceback = np.zeros((len_s1 + 1, len_s2 + 1), dtype=int)

    #Remplissage de la matrice
    for i in range(1, len_s1 + 1):
        for j in range(1, len_s2 + 1):
            #Calcul de chaque condition
            diag = mat_score[i-1,j-1] + init_mat[alphabet.index(S1[i-1]),alphabet.index(S2[j-1])] 
            deletion = mat_score[i-1,j] + (extension if i > 1 else ouverture)
            insertion = mat_score[i,j-1] + (extension if i > 1 else ouverture)
            
            #Récupération du max
            sc_max = max(0,diag,deletion,insertion)
            mat_score[i,j] = sc_max
            
            if sc_max == diag:
                mat_traceback[i,j]=0
            elif sc_max == deletion:
                mat_traceback[i,j]=1
            else: 
                mat_traceback[i,j]=2

    #Traceback
    
    #Récupération de max et de ses positions
    sc_max = np.max(mat_score)
    max_pos = np.argwhere(mat_score==sc_max)
    
    #Initialisation des listes et arrays nécessaires
    list_alignements = []
    
    #Parcours sur le nombre de positions trouvés
    for p in max_pos:
        alig_s1 = np.empty((0,),dtype=str)
        alig_s2 = np.empty((0,),dtype=str)
        
        l1,l2 = p
    
        #Parcours en reverse
        while mat_score[l1,l2] != 0:
            #Test si diagonale
            if mat_traceback[l1,l2] == 0 :
                alig_s1 = np.append(alig_s1,S1[l1-1])
                alig_s2 = np.append(alig_s2,S2[l2-1])
                l1 -= 1
                l2 -= 1

            #Test si délétion
            elif mat_traceback[l1,l2] == 1 :
                alig_s1 = np.append(alig_s1,S1[l1-1])
                alig_s2 = np.append(alig_s2,'-')
                l1 -= 1

            #Test si insertion
            else :
                alig_s1 = np.append(alig_s1,'-')
                alig_s2 = np.append(alig_s2,S2[l2-1])
                l2 -= 1            

        #Renversement des arrays
        alig_s1 = np.flip(alig_s1)
        alig_s2 = np.flip(alig_s2)

        #Ajout à la liste des alignements
        list_alignements.append([''.join(alig_s1),''.join(alig_s2)])

    return list_alignements, mat_score[len_s1,len_s2]
   

2. Comparer le résultat des deux algorithmes sur les séquences données en TD.

In [204]:
#Exécution de SW
align_sw,score_sw,mat_traceback,ms = SW(S1,S2,init_mat,ouverture,extension,alphabet)

print(f"\nAlignement obtenu après SW avec match = {match}, mismatch = {mismatch}, ouverture de gap = {ouverture} et extension de gap = {extension}\n")
print(f"Score : {score_sw}\n")
for al in align_sw:
    print(f"Séquence 1 : {al[0]}\nSéquence 2 : {al[1]}\n")


Alignement obtenu après SW avec match = 1, mismatch = -1, ouverture de gap = -1 et extension de gap = -1

Score : 3

Séquence 1 : TGA
Séquence 2 : TGA

Séquence 1 : TG-AC
Séquence 2 : TGAAC



L'alignement le plus grand à un meilleur score que l'aligenement global.

In [211]:
align_grand_sw,score_grand_sw = SW(S_ABL,S_10PK,init_mat_aa,ouverture_grand,extension_grand,alphabet_aa)

print(f"\nAlignement obtenu après SW avec match = {match_grand}, mismatch = {mismatch_grand}, ouverture de gap = {ouverture_grand} et extension de gap = {extension_grand}\n")
print(f"Score : {score_grand_sw}\n")
for al in align_grand_sw:
    print(f"Séquence 1 : {al[0]}\n\nSéquence 2 : {al[1]}\n")


Alignement obtenu après SW avec match = 1, mismatch = -1, ouverture de gap = -11 et extension de gap = -1

Score : 0

Séquence 1 : GPSENDPNLFVALYDFVASGDNTLSITKGEKLRVLGYNHNGEWCEAQTKNGQGWVPSNYITPVNSLEKHSWYHGPVSRNAAEYLLSSGINGSFLVRESESSPGQRSISLRYEGRVYHYRINTASDGKLYVSSESRFNTLAELVHHHSTVADGLITTLHYPAP

Séquence 2 : GPSENDPNLFVALYDFVASGDNTLSITKGEKLRVLGYNHNGEWCEAQTKNGQGWVPSNYITPVNSLEKHSWYHGPVSRNAAEYLLSSGINGSFLVRESESSPGQRSISLRYEGRVYHYRINTASDGKLYVSSESRFNTLAELVHHHSTVADGLITTLHYPAP



Avec les séquences de la partie 2, on retrouve les mêmes séquences qu'avait obtenu BLAST.